In [96]:
import sys
import os
# caution: path[0] is reserved for script path (or '' in REPL).
sys.path.insert(1, os.path.abspath('./../src'))

import IRBEM
import data_loader
import datetime
from cdflib.epochs_astropy import CDFAstropy as cdfepoch
import numpy as np
import chorus_machine_learning_helper



In [97]:
start_time = datetime.datetime(year = 1998, month = 1, day = 1, hour = 1, minute = 1, second = 1)
end_time = datetime.datetime(year = 1999, month = 1, day = 1, hour = 1, minute = 1, second = 1)

In [ ]:

model = IRBEM.MagFields(path = r".\..\IRBEM\libirbem.dll",
                        options = [1,0,0,0,0], 
                        kext = "T89",
                        sysaxes = 0, #GDZ
                        verbose = True)

OMNI = data_loader.load_raw_data_from_config(id = ["OMNI", "ONE_HOUR_RESOLUTION"],
                                             start = start_time,
                                             end = end_time)

time = cdfepoch.unixtime(OMNI["Epoch"])
Kp = OMNI["KP"]

In [104]:
year = 1998
start_of_year_UNIX_TIME = datetime.datetime(year = year, month = 1, day = 1).timestamp()
end_of_year_UNIX_TIME = datetime.datetime(year = year + 1, month = 1, day = 1).timestamp()
equally_spaced_minutes = np.arange(start = start_of_year_UNIX_TIME, stop = end_of_year_UNIX_TIME + 60, step = 60)

MPE = chorus_machine_learning_helper.load_MPE_year(year)

Finished loading POES data for year : 1998


In [105]:
sat = MPE[0]

unix_times_of_averages = []
avg_geog_lat = []
avg_geog_lon = []

for MINUTE in equally_spaced_minutes:
    
    TIME_RANGE = np.searchsorted(a = sat["UNIX_TIME"], v = [(MINUTE - 30), (MINUTE + 30)])
    
    if (TIME_RANGE[0] != TIME_RANGE[1]):
        
        unix_times_of_averages.append(MINUTE)    
        avg_geog_lat.append(np.nanmean(sat["geogLat"][TIME_RANGE[0] : TIME_RANGE[1]]))
        y_of_lon = np.nanmean(np.sin(np.deg2rad(sat["geogLon"][TIME_RANGE[0] : TIME_RANGE[1]])))
        x_of_lon = np.nanmean(np.cos(np.deg2rad(sat["geogLon"][TIME_RANGE[0] : TIME_RANGE[1]])))            
        avg_geog_lon.append(np.mod((np.rad2deg(np.arctan2(y_of_lon, x_of_lon)) + 360), 360))

In [106]:
KP_INTERPOLATED = np.interp(unix_times_of_averages, xp = time, fp = Kp, left=np.nan, right=np.nan).astype(np.float64)

In [107]:
points = np.array([(unix_times_of_averages[i], 817, avg_geog_lat[i], avg_geog_lon[i], KP_INTERPOLATED[i]) for i in range(len(KP_INTERPOLATED))])
dates = [datetime.datetime.fromtimestamp(unix_times_of_averages[i]) for i in range(len(unix_times_of_averages))]

print(points.shape)

coords = {
    "x1" : points[:25, 1],
    "x2" : points[:25, 2],
    "x3" : points[:25, 3],
    "dateTime" : dates[:25]
}

mag_inputs = {
    "Kp" : KP_INTERPOLATED[:25]
}

print(coords)

model.make_lstar(X = coords, maginput = mag_inputs)

(214122, 5)
{'x1': array([817., 817., 817., 817., 817., 817., 817., 817., 817., 817., 817.,
       817., 817., 817., 817., 817., 817., 817., 817., 817., 817., 817.,
       817., 817., 817.]), 'x2': array([-18.23640013, -14.47854948, -11.81563314,  -7.89704967,
        -4.1329748 ,  -1.30914995,   2.92729986,   6.22227478,
         9.98727489,  13.28026613,  16.57139969,  20.32972527,
        24.08417463,  27.36526616,  30.64169979,  34.37982464,
        38.10979843,  41.36563238,  44.61224842,  48.30984783,
        51.99032307,  55.19343313,  58.37449741,  75.27612495,
        72.10187531]), 'x3': array([295.16536712, 294.27519226, 293.65886471, 292.76546478,
       291.91870117, 291.28744507, 290.34155273, 289.60281372,
       288.75057984, 287.99417114, 287.22281647, 286.31859589,
       285.38284303, 284.53241984, 283.6426697 , 282.57037356,
       281.42317967, 280.34506227, 279.17397317, 277.70134749,
       276.04101585, 274.39277149, 272.48529885, 139.41741083,
       131.788975

{'Lm': [1.2107445394316128,
  1.1992348540344557,
  1.196464382343141,
  1.201606311905166,
  1.2166448677845585,
  1.2345656637177185,
  1.2723851037307035,
  1.3120073622866408,
  1.3690913797825408,
  1.4311615043740196,
  1.5066586422733312,
  1.6131137338538268,
  1.7466377175991437,
  1.8917995526800107,
  2.0709493987428504,
  2.330350625476344,
  2.668494389129815,
  3.053424145908943,
  3.5544331850504065,
  4.330681200529286,
  5.4203056287923275,
  6.712727725275359,
  -8.318934522849533,
  14.63577395351885,
  8.563554021885109],
 'MLT': [14.817562748892424,
  14.783224943493419,
  14.764160747400101,
  14.727285785015187,
  14.691950339054017,
  14.669302992439395,
  14.625048554367712,
  14.593189192723532,
  14.552318035905582,
  14.516766692723866,
  14.478858566990729,
  14.42958738682374,
  14.375857605843397,
  14.326657913630232,
  14.272204146762014,
  14.199769284606823,
  14.116922677348054,
  14.035917544810188,
  13.941473528191533,
  13.810113809903612,
  13.6